In [9]:
import os
import glob
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TOGETHER_API_KEY"] = os.getenv("TOGETHER_API_KEY")
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="gemma2-9b-it")

In [21]:
from langchain_community.tools import DuckDuckGoSearchRun
web_search = DuckDuckGoSearchRun()
web_search.run("Who are the 11 players represent team india in england test series of 4th match?")

d:\LangGraph\langgraph-handson\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


"Jul 15, 2025 · Learn & Understand all information about England including India tour of England 2025 Squad, Player Details & More at ESPNcricinfo. 1 day ago · India and England will clash once again in the fourth Test at Old Trafford, Manchester, starting on July 23 (Wednesday). England lead the series 2-1, following their 22-run win in the … Jul 10, 2025 · Here are the full details of the England vs India 4th Test Match 2025 Timing, Captain, Squad or Players List. When is ENG vs IND, 4th Test? ENG vs IND, 4th Test to be … 1 day ago · After India's heartbreaking loss at the Home of Cricket, Lord's, the Anderson-Tendulkar trophy now moves on to the Old Trafford Stadium in Manchester for the 4th Test of … 19 hours ago · India vs England 4th Test, India playing 11 vs England, India vs England team news, Arshdeep Singh injury update, Nitish Reddy news, India vs England 4th Test squad, …"

In [22]:
from langchain_experimental.utilities import PythonREPL
repl = PythonREPL()

code="""
x = 10
y = 20
print(f"The sum of {x} and {y} is {x + y}")
"""
repl.run(code)

'The sum of 10 and 20 is 30\n'

In [23]:
from langchain_core.tools import tool
from typing import Annotated, Literal

@tool
def python_repl_tool(code: Annotated[str, "The python code to execute to generate chart"]):
    """Run Python code and return the output."""

    try:
        result = repl.run(code)
    except Exception as e:
        return f"Error executing code: {e}"
    
    result_str = f"Sucessfully executed code: Output:{result}"

    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [24]:
def make_system_prompt(instruction:str)->str:
    return  (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{instruction}"
    )

make_system_prompt("You can only do research. You are working with a chart generator colleague.")

"You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to progress towards answering the question. If you are unable to fully answer, that's OK, another assistant with different tools  will help where you left off. Execute what you can to make progress. If you or any of the other assistants have the final answer or deliverable, prefix your response with FINAL ANSWER so the team knows to stop.\nYou can only do research. You are working with a chart generator colleague."

In [25]:
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph,START,END

def get_next_node(last_message:BaseMessage, goto:str):
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return goto


In [26]:
from langgraph.prebuilt import create_react_agent
research_agent = create_react_agent(
    llm,
    tools=[web_search, python_repl_tool],
    prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague."
    ),
)

chart_agent = create_react_agent(
    llm,
    [python_repl_tool],
    prompt=make_system_prompt(
        "You can only generate charts. You are working with a researcher colleague."
    ),
)


In [27]:
from langgraph.types import Command

def research_node(state:MessagesState)->Command[Literal["chart_generator", END]]:
    
    result = research_agent.invoke(state)
    
    goto = get_next_node(result["messages"][-1], "chart_generator")
    
    result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="researcher")
    
    return Command(update={"messages": result["messages"]},goto=goto)


def chart_node(state:MessagesState)-> Command[Literal["researcher", END]]:
    
    result = chart_agent.invoke(state)
    
    goto = get_next_node(result["messages"][-1], "researcher")
    
    result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="chart_generator")
    
    return Command(update={"messages": result["messages"]},goto=goto)

In [28]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(MessagesState)
workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)

workflow.add_edge(START, "researcher")
graph = workflow.compile()


In [29]:
graph.invoke({"messages": [("user","get the UK's GDP over the past 3 years, then make a line chart of it.Once you make the chart, finish.")],})

d:\LangGraph\langgraph-handson\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
Task researcher with path ('__pregel_pull', 'researcher') wrote to unknown channel branch:to:__end__, ignoring it.


{'messages': [HumanMessage(content="get the UK's GDP over the past 3 years, then make a line chart of it.Once you make the chart, finish.", additional_kwargs={}, response_metadata={}, id='57c1c781-86b3-4802-97cb-c604dbe82b81'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '86p23rz7g', 'function': {'arguments': '{"query":"UK GDP over the past 3 years"}', 'name': 'duckduckgo_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 1225, 'total_tokens': 1314, 'completion_time': 0.161818182, 'prompt_time': 0.023355891, 'queue_time': 0.258807022, 'total_time': 0.185174073}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--591afc74-a5db-48d3-b70c-b77f9879e49c-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'UK GDP over the past 3 years'}, 'id': '86p23rz7g', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1225, 'output_t